### Creating an NLP pipeline

In [1]:
import nltk
import threading
import queue
import feedparser
import uuid

In [2]:
threads = []
import warnings
warnings.filterwarnings("ignore")

In [3]:
queues = [queue.Queue(), queue.Queue()]

In [4]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

In [5]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)

In [6]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()

In [7]:
def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)

    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()

In [8]:
if __name__ == '__main__':
    runProgram()

>> 591efaf0-5a01-4dd2-8471-934b2923abc3 : 'Raazi' song 'Ae Watan' will evoke patriotism
>> ca28a101-2c7f-4271-9f5f-44f947134bc8 : 2 States @ 4: Interesting facts
>> 016e3c34-e58b-4a65-888a-9a3455cd0b10 : Mouni Roy in Salman Khan's 'Dabangg 3'?
  << 591efaf0-5a01-4dd2-8471-934b2923abc3 : (GPE Watan/NNP), 
  << ca28a101-2c7f-4271-9f5f-44f947134bc8 : 
  << 016e3c34-e58b-4a65-888a-9a3455cd0b10 : (PERSON Mouni/NNP), (ORGANIZATION Roy/NNP), (GPE Salman/NNP Khan/NNP), 


### Solving the text similarity problem

In [9]:
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
class TextSimilarityExample:
    def __init__(self):
        self.statements = [
            'ruled india',
            'Chalukyas ruled Badami',
            'So many kingdoms ruled India',
            'Lalbagh is a botanical garden in India'
        ]
    def TF(self, sentence):
        words = nltk.word_tokenize(sentence.lower())
        freq = nltk.FreqDist(words)
        dictionary = {}
        for key in freq.keys():
            norm = freq[key]/float(len(words))
            dictionary[key] = norm
        return dictionary

    def IDF(self):
        def idf(TotalNumberOfDocuments, NumberOfDocumentsWithThisWord):
            return 1.0 + math.log(TotalNumberOfDocuments/NumberOfDocumentsWithThisWord)
        numDocuments = len(self.statements)
        uniqueWords = {}
        idfValues = {}
        for sentence in self.statements:
            for word in nltk.word_tokenize(sentence.lower()):
                if word not in uniqueWords:
                    uniqueWords[word] = 1
                else:
                    uniqueWords[word] += 1
        for word in uniqueWords:
            idfValues[word] = idf(numDocuments, uniqueWords[word])
        return idfValues

    def TF_IDF(self, query):
        words = nltk.word_tokenize(query.lower())
        idf = self.IDF()
        vectors = {}
        for sentence in self.statements:
            tf = self.TF(sentence)
            for word in words:
                tfv = tf[word] if word in tf else 0.0
                idfv = idf[word] if word in idf else 0.0
                mul = tfv * idfv
                if word not in vectors:
                    vectors[word] = []
                vectors[word].append(mul)
        return vectors

    def displayVectors(self, vectors):
        print(self.statements)
        for word in vectors:
            print("{} -> {}".format(word, vectors[word]))

    def cosineSimilarity(self):
        vec = TfidfVectorizer()
        matrix = vec.fit_transform(self.statements)
        for j in range(1, 5):
            i = j - 1
            print("\tsimilarity of document {} with others".format(i))
            similarity = cosine_similarity(matrix[i:j], matrix)
            print(similarity)

    def demo(self):
        inputQuery = self.statements[0]
        vectors = self.TF_IDF(inputQuery)
        self.displayVectors(vectors)
        self.cosineSimilarity()

In [11]:
similarity = TextSimilarityExample()
similarity.demo()

['ruled india', 'Chalukyas ruled Badami', 'So many kingdoms ruled India', 'Lalbagh is a botanical garden in India']
ruled -> [0.6438410362258904, 0.42922735748392693, 0.2575364144903562, 0.0]
india -> [0.6438410362258904, 0.0, 0.2575364144903562, 0.18395458177882582]
	similarity of document 0 with others
[[ 1.          0.29088811  0.46216171  0.19409143]]
	similarity of document 1 with others
[[ 0.29088811  1.          0.13443735  0.        ]]
	similarity of document 2 with others
[[ 0.46216171  0.13443735  1.          0.08970163]]
	similarity of document 3 with others
[[ 0.19409143  0.          0.08970163  1.        ]]


### Resolving anaphora

In [12]:
from nltk.chunk import tree2conlltags
from nltk.corpus import names
import random

In [13]:
class AnaphoraExample:
    def __init__(self):
        males = [(name, 'male') for name in names.words('male.txt')]
        females = [(name, 'female') for name in names.words('female.txt')]
        combined = males + females
        random.shuffle(combined)
        training = [(self.feature(name), gender) for (name, gender) in combined]
        self._classifier = nltk.NaiveBayesClassifier.train(training)

    def feature(self, word):
        return {'last(1)' : word[-1]}

    def gender(self, word):
        return self._classifier.classify(self.feature(word))

    def learnAnaphora(self):
        sentences = [
            "John is a man. He walks",
            "John and Mary are married. They have two kids",
            "In order for Ravi to be successful, he should follow John",
            "John met Mary in Barista. She asked him to order a Pizza"
        ]

        for sent in sentences:
            chunks = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)), binary=False)
            stack = []
            print(sent)
            items = tree2conlltags(chunks)
            for item in items:
                if item[1] == 'NNP' and (item[2] == 'B-PERSON' or item[2] == 'O'):
                    stack.append((item[0], self.gender(item[0])))
                elif item[1] == 'CC':
                    stack.append(item[0])
                elif item[1] == 'PRP':
                    stack.append(item[0])
            print("\t {}".format(stack))

In [14]:
anaphora = AnaphoraExample()
anaphora.learnAnaphora()

John is a man. He walks
	 [('John', 'male'), 'He']
John and Mary are married. They have two kids
	 [('John', 'male'), 'and', ('Mary', 'female'), 'They']
In order for Ravi to be successful, he should follow John
	 [('Ravi', 'female'), 'he', ('John', 'male')]
John met Mary in Barista. She asked him to order a Pizza
	 [('John', 'male'), ('Mary', 'female'), 'She', 'him']


### Disambiguating word sense

In [15]:
def understandWordSenseExamples():
    words = ['wind', 'date', 'left']
    print("-- examples --")
    for word in words:
        syns = nltk.corpus.wordnet.synsets(word)
        for syn in syns[:2]:
            for example in syn.examples()[:2]:
                print("{} -> {} -> {}".format(word, syn.name(), example))

In [16]:
def understandBuiltinWSD():
    print("-- built-in wsd --")
    maps = [
        ('Is it the fish net that you are using to catch fish ?', 'fish', 'n'),
        ('Please dont point your finger at others.', 'point', 'n'),
        ('I went to the river bank to see the sun rise', 'bank', 'n'),
    ]
    for m in maps:
        print("Sense '{}' for '{}' -> '{}'".format(m[0], m[1], nltk.wsd.lesk(m[0], m[1], m[2])))

In [17]:
if __name__ == '__main__':
    understandWordSenseExamples()
    understandBuiltinWSD()

-- examples --
wind -> wind.n.01 -> trees bent under the fierce winds
wind -> wind.n.01 -> when there is no wind, row
wind -> wind.n.02 -> the winds of change
date -> date.n.01 -> what is the date today?
date -> date.n.02 -> his date never stopped talking
left -> left.n.01 -> she stood on the left
-- built-in wsd --
Sense 'Is it the fish net that you are using to catch fish ?' for 'fish' -> 'Synset('pisces.n.02')'
Sense 'Please dont point your finger at others.' for 'point' -> 'Synset('point.n.25')'
Sense 'I went to the river bank to see the sun rise' for 'bank' -> 'Synset('savings_bank.n.02')'


### Performing sentiment analysis

In [18]:
import nltk.sentiment.sentiment_analyzer

In [19]:
def wordBasedSentiment():
    positive_words = ['love', 'hope', 'joy']
    text = 'Rainfall this year brings lot of hope and joy to Farmers.'.split()
    analysis = nltk.sentiment.util.extract_unigram_feats(text, positive_words)
    print(' -- single word sentiment --')
    print(analysis)

In [20]:
def multiWordBasedSentiment():
    word_sets = [('heavy', 'rains'), ('flood', 'bengaluru')]
    text = 'heavy rains cause flash flooding in bengaluru'.split()
    analysis = nltk.sentiment.util.extract_bigram_feats(text, word_sets)
    print(' -- multi word sentiment --')
    print(analysis)

In [21]:
def markNegativity():
    text = 'Rainfall last year did not bring joy to Farmers'.split()
    negation = nltk.sentiment.util.mark_negation(text)
    print(' -- negativity --')
    print(negation)

In [22]:
if __name__ == '__main__':
    wordBasedSentiment()
    multiWordBasedSentiment()
    markNegativity()

 -- single word sentiment --
{'contains(love)': False, 'contains(hope)': True, 'contains(joy)': True}
 -- multi word sentiment --
{'contains(heavy - rains)': True, 'contains(flood - bengaluru)': False}
 -- negativity --
['Rainfall', 'last', 'year', 'did', 'not', 'bring_NEG', 'joy_NEG', 'to_NEG', 'Farmers_NEG']


### Exploring advanced sentiment analysis

In [24]:
import nltk.sentiment.util
import nltk.sentiment.sentiment_analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [25]:
def mySentimentAnalyzer():
    def score_feedback(text):
        positive_words = ['love', 'genuine', 'liked']
        if '_NEG' in ' '.join(nltk.sentiment.util.mark_negation(text.split())):
            score = -1
        else:
            analysis = nltk.sentiment.util.extract_unigram_feats(text.split(), positive_words)
            if True in analysis.values():
                score = 1
            else:
                score = 0
        return score

    feedback = """I love the items in this shop, very genuine and quality is well maintained.
    I have visited this shop and had samosa, my friends liked it very much.
    ok average food in this shop.
    Fridays are very busy in this shop, do not place orders during this day."""
    print(' -- custom scorer --')
    for text in feedback.split("\n"):
        print("score = {} for >> {}".format(score_feedback(text), text))

In [26]:
def advancedSentimentAnalyzer():
    sentences = [
        ':)',
        ':(',
        'She is so :(',
        'I love the way cricket is played by the champions',
        'She neither likes coffee nor tea',
    ]
    senti = SentimentIntensityAnalyzer()
    print(' -- built-in intensity analyser --')
    for sentence in sentences:
        print('[{}]'.format(sentence), end=' --> ')
        kvp = senti.polarity_scores(sentence)
        for k in kvp:
            print('{} = {}, '.format(k, kvp[k]), end='')
        print()

In [27]:
if __name__ == '__main__':
    advancedSentimentAnalyzer()
    mySentimentAnalyzer()

 -- built-in intensity analyser --
[:)] --> pos = 1.0, neg = 0.0, compound = 0.4588, neu = 0.0, 
[:(] --> pos = 0.0, neg = 1.0, compound = -0.4404, neu = 0.0, 
[She is so :(] --> pos = 0.0, neg = 0.555, compound = -0.5777, neu = 0.445, 
[I love the way cricket is played by the champions] --> pos = 0.625, neg = 0.0, compound = 0.875, neu = 0.375, 
[She neither likes coffee nor tea] --> pos = 0.0, neg = 0.318, compound = -0.3252, neu = 0.682, 
 -- custom scorer --
score = 1 for >> I love the items in this shop, very genuine and quality is well maintained.
score = 1 for >>     I have visited this shop and had samosa, my friends liked it very much.
score = 0 for >>     ok average food in this shop.
score = -1 for >>     Fridays are very busy in this shop, do not place orders during this day.


### Creating a conversational assistant or chatbot

In [28]:
def builtinEngines(whichOne):
    if whichOne == 'eliza':
        nltk.chat.eliza.demo()
    elif whichOne == 'iesha':
        nltk.chat.iesha.demo()
    elif whichOne == 'rude':
        nltk.chat.rude.demo()
    elif whichOne == 'suntsu':
        nltk.chat.suntsu.demo()
    elif whichOne == 'zen':
        nltk.chat.zen.demo()
    else:
        print("unknown built-in chat engine {}".format(whichOne))

In [29]:
def myEngine():
    chatpairs = (
        (r"(.*?)Stock price(.*)",
            ("Today stock price is 100",
            "I am unable to find out the stock price.")),
        (r"(.*?)not well(.*)",
            ("Oh, take care. May be you should visit a doctor",
            "Did you take some medicine ?")),
        (r"(.*?)raining(.*)",
            ("Its monsoon season, what more do you expect ?",
            "Yes, its good for farmers")),
        (r"How(.*?)health(.*)",
            ("I am always healthy.",
            "I am a program, super healthy!")),
        (r".*",
            ("I am good. How are you today ?",
            "What brings you here ?"))
    )
    def chat():
        print("!"*80)
        print(" >> my Engine << ")
        print("Talk to the program using normal english")
        print("="*80)
        print("Enter 'quit' when done")
        chatbot = nltk.chat.util.Chat(chatpairs, nltk.chat.util.reflections)
        chatbot.converse()

    chat()

In [30]:
if __name__ == '__main__':
    for engine in ['eliza', 'iesha', 'rude', 'suntsu', 'zen']:
        print("=== demo of {} ===".format(engine))
        builtinEngines(engine)
        print()
    myEngine()

=== demo of eliza ===
Therapist
---------
Talk to the program by typing in plain English, using normal upper-
and lower-case letters and punctuation.  Enter "quit" when done.
Hello.  How are you feeling today?
>Nice
I see.  And what does that tell you?
>quit
Thank you for talking with me.

=== demo of iesha ===
Iesha the TeenBoT
---------
Talk to the program by typing in plain English, using normal upper-
and lower-case letters and punctuation.  Enter "quit" when done.
hi!! i'm iesha! who r u??!
>quit
mom says i have to go eat dinner now :,( bye!!

=== demo of rude ===
Talk to the program by typing in plain English, using normal upper-
and lower-case letters and punctuation.  Enter "quit" when done.
I suppose I should say hello.
>quit
Either become more thrilling or get lost, buddy.

=== demo of suntsu ===
Talk to the program by typing in plain English, using normal upper-
and lower-case letters and punctuation.  Enter "quit" when done.
You seek enlightenment?
>quit
Plan well

=== demo